In [0]:
scope = "kv-cerveja"
key = "cervejas"
storage_account_name = "stgbeer"
container_name = "cervejas"
mount_point = '/mnt/cervejas'


if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
  
  connect_str = dbutils.secrets.get(scope=scope, key=key)

  dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point=mount_point,
    extra_configs={
      f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": connect_str
    }
  )
  print(f"Mounted {mount_point}")
else:
  print(f"{mount_point} já está montado")

/mnt/cervejas já está montado


In [0]:
%fs ls /mnt/cervejas

path,name,size,modificationTime
dbfs:/mnt/cervejas/bronze/,bronze/,0,0
dbfs:/mnt/cervejas/ouro/,ouro/,0,0
dbfs:/mnt/cervejas/prata/,prata/,0,0


In [0]:
import requests
import json

url = 'https://api.openbrewerydb.org/v1/breweries'
cervejas = []
page = 1

while True:
    response = requests.get(f'{url}?page={page}&per_page=50')
    data = json.loads(response.text)
    if not data:
        break
    cervejas.extend(data)
    page += 1
print(f'o arquivo contem {len(cervejas)} linhas')



o arquivo contem 8257 linhas


In [0]:
df = spark.createDataFrame(cervejas)

output_path = "/mnt/cervejas/bronze/cervejas"

df.write.mode("overwrite").json(output_path)

print("Dados salvos com sucesso na camada bronze")

Dados salvos com sucesso na camada bronze
